## 训练句子相似度比较模型

In [ ]:
import numpy as np
import pandas as pd
import torch
from gensim.models.word2vec import LineSentence
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [18]:
# 翻译爬虫
!pip install googletrans

  Created wheel for googletrans: filename=googletrans-2.4.0-cp36-none-any.whl size=15789 sha256=dcc72d2b75ba9fa4de2de8fbd9cb658629f6a0dd23727622280404df67ec72fe
  Stored in directory: C:\Users\Raytine\AppData\Local\pip\Cache\wheels\50\d6\e7\a8efd5f2427d5eb258070048718fa56ee5ac57fd6f53505f95
Successfully built googletrans


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [29]:
from googletrans import Translator
translator = Translator(service_urls=[
      'translate.google.cn'
])

In [2]:
# load corpus
rootdir = "E:/workspace/py/NLPHomework/Assignment-04/out/"
fr=open(rootdir+'sentences','r',encoding='utf-8')
sentences = LineSentence(fr)

In [4]:
import sys
from os import path
sys.path.append( './chinese_bert_similarity')
from chinese_bert_similarity.bert_sim import BertSim

bs = BertSim(gpu_no=0, log_dir='chinese_bert_similarity/log/', bert_sim_dir='chinese_bert_similarity/bert_sim_model/', verbose=True)

INFO:tensorflow:Restoring parameters from chinese_bert_similarity/bert_sim_model/model.ckpt-6123


In [78]:
# 1. 数据集中加入翻译数据作为高相似度的正样本
en_data = []
with open('news-commentary-v13.zh-en.en','r',encoding='utf-8') as en_file:
    for line in en_file.readlines():
        if '&' in line:
            line = line.replace('\n','').replace('&nbsp;',' ').replace('&amp;','&').replace('&#45;','-').replace('&#160;',' ') \
                .replace('&hellip;','…').replace('&quot;','"')
        en_data.append(line)
print('en finish')
zh_data = []
with open('news-commentary-v13.zh-en.zh','r',encoding='utf-8') as zh_file:
    for line in zh_file.readlines():
        zh_data.append(line.replace('\n',''))
sim = [1 for _ in zh_data]

en finish


In [20]:
sim_data.head()

,en,zh,sim
0,1929 or 1989?,1929年还是1989年?,1
1,PARIS – As the economic crisis deepens and wid...,巴黎-随着经济危机不断加深和蔓延，整个世界一直在寻找历史上的类似事件希望有助于我们了解目前正...,1
2,"At the start of the crisis, many people likene...",一开始，很多人把这次危机比作1982年或1973年所发生的情况，这样得类比是令人宽心的，因为...,1
3,"Today, the mood is much grimmer, with referenc...",如今人们的心情却是沉重多了，许多人开始把这次危机与1929年和1931年相比，即使一些国家政...,1
4,The tendency is either excessive restraint (Eu...,目前的趋势是，要么是过度的克制（欧洲），要么是努力的扩展（美国）。,1


In [ ]:
import time
# trans_data = []
trans_data_continue = []
# translations = translator.translate(en_data, dest='zh-CN',src='en')
# for translation in translations:
#     trans_data.append(translation.text)
for sentence in en_data[27942:]:
    try:
        translation = translator.translate([sentence], dest='zh-CN',src='en')
        time.sleep(1)
    #     print(translation[0].text)
        trans_data_continue.append(translation[0].text)
    except:
        print(sentence)
        break

In [75]:
print(len(trans_data_continue))

14472


In [76]:
for translation in trans_data_continue:
    trans_data.append(translation)

In [77]:
print(len(trans_data))

27942


In [72]:
sim_data = pd.DataFrame({'translation':trans_data,'zh':zh_data[:27942],'sim':sim[:27942]})
sim_data.to_csv('sim_data.csv')

In [73]:
sim_data.head()

,translation,zh,sim
0,1929年或1989年？,1929年还是1989年?,1
1,巴黎 - 随着经济危机的加深和加宽，世界一直在寻找历史类比来帮助我们理解正在发生的事情。,巴黎-随着经济危机不断加深和蔓延，整个世界一直在寻找历史上的类似事件希望有助于我们了解目前正...,1
2,在危机开始时，很多人把它比做1982年或1973年，这是令人欣慰的，因为这两个日期均指传统的...,一开始，很多人把这次危机比作1982年或1973年所发生的情况，这样得类比是令人宽心的，因为...,1
3,今天，心情是非常严峻，与开始的比比皆是，即使有些国家的政府继续表现得好像危机较特殊的更经典的...,如今人们的心情却是沉重多了，许多人开始把这次危机与1929年和1931年相比，即使一些国家政...,1
4,目前的趋势是要么过度限制（欧洲）或努力的扩散（美国）。,目前的趋势是，要么是过度的克制（欧洲），要么是努力的扩展（美国）。,1


In [ ]:
# 2. 打乱文本顺序,生成不相似数据
import copy
trans_data_2 = copy.deepcopy(trans_data)
zh_2 = copy.deepcopy(zh_data)

import random
random.shuffle(trans_data_2)
random.shuffle(zh_2)
sim_2 = []
for idx in range(len(trans_data_2)):
    trans = trans_data_2[idx]
    zh = zh_2[idx]
    sim_2.append(bs.predict(trans,zh))
    
trans_data += trans_data_2
zh_data += zh_2
sim += sim_2

random.shuffle(trans_data)
random.shuffle(zh_data)
random.shuffle(sim)

In [ ]:
sim_data = pd.DataFrame({'translation':trans_data,'zh':zh_data,'sim':sim})
sim_data.to_csv('sim_data.csv')
data = sim_data.sample(frac=1)

#### 数据预处理完毕

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from typing import Tuple, List
import re

Texts = torch.LongTensor

class SentenceDataset(Dataset):
    def __init__(self, tokenizer, dataframe, device):
        self.device = device
        self.tokenizer = tokenizer
        self.pad_idx = tokenizer.pad_token_id
        self.X = []
        self.Y = []
        for i, (row) in tqdm(dataframe.iterrows()):
            if len(tokenizer.tokenize(row["zh"])) > 120:
                continue
            # 移除第一句标点符号
            sentence1 = re.sub(r'[。，?]','',row["zh"])
            orignal_text = tokenizer.encode(sentence1)
#             orignal_text = torch.LongTensor(text)
            # 移除第二句标点符号
            sentence2 = re.sub(r'[。，?]','',row["translation"])
#             print(sentence2)
            trans_text = tokenizer.encode(sentence2, add_special_tokens=True)
#             print(tokenizer.decode(text[1:]))
            text = torch.LongTensor(orignal_text+trans_text)
            
            tags = torch.FloatTensor(row[["sim"]])
            self.X.append(text)
            self.Y.append(tags)
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, index: int) -> Tuple[Texts, torch.LongTensor]:
        return self.X[index], self.Y[index]

In [ ]:
tokenizer = BertTokenizer.from_pretrained('./model')

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(data, test_size=0.05)
print(len(train_df))
print(len(val_df))

In [ ]:
def collate_fn(batch: List[Tuple[Texts, torch.LongTensor]]) \
        -> Tuple[Texts, torch.LongTensor]:
    x, y = list(zip(*batch))
    x = pad_sequence(x, batch_first=True, padding_value=0)
    y = torch.stack(y)
    return x.to(device), y.to(device)

train_dataset = SentenceDataset(tokenizer, train_df, device)
dev_dataset = SentenceDataset(tokenizer, val_df, device)

BATCH_SIZE = 8
train_sampler = RandomSampler(train_dataset)
dev_sampler = RandomSampler(dev_dataset)
train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler, collate_fn=collate_fn)
dev_iterator = DataLoader(dev_dataset, batch_size=BATCH_SIZE, sampler=dev_sampler, collate_fn=collate_fn)

In [ ]:
# . 训练bert下游模型
class BertSimilarity(BertPreTrainedModel):
    def __init__(self, config):
        super(BertSimilarity, self).__init__(config)
        self.bert = BertModel(config)
        self.similarity = nn.Linear(config.hidden_size, 2)
        
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
            labels=None):
        outputs = self.bert(input_ids,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask)
        cls_output = outputs[1] # batch, hidden
        cls_output = self.similarity(cls_output) # batch, 2
#         print(cls_output.shape)
        cls_output = torch.sigmoid(cls_output)
        criterion = nn.BCELoss()
#         print(cls_output.shape,labels.shape)
        loss = 0
        if labels is not None:
            loss = criterion(cls_output, labels)
        return loss, cls_output

# model = BertModel.from_pretrained('hfl/chinese-bert-wwm-ext')
model = BertSimilarity.from_pretrained('./model').to(device)

In [ ]:
from sklearn.metrics import r2_score
def train(model, iterator, optimizer, scheduler):
    model.train()
    total_loss = 0
    for x, y in tqdm(iterator):
        optimizer.zero_grad()
        mask = (x != 0).float()
#         print(x.shape,y.shape)
        loss, outputs = model(x, attention_mask=mask, labels=y)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
    print(f"Train loss {total_loss / len(iterator)}")

def evaluate(model, iterator):
    model.eval()
    pred = []
    true = []
    with torch.no_grad():
        total_loss = 0
        for x, y in tqdm(iterator):
            mask = (x != 0).float()
            loss, outputs = model(x, attention_mask=mask, labels=y)
            total_loss += loss
            true += y.cpu().numpy().tolist()
            pred += outputs.cpu().numpy().tolist()
    true = np.array(true)
    pred = np.array(pred)
    print(true.shape,pred.shape)
    for i, name in enumerate(['non-similarity','similarity']):
        print(f"{name} roc_auc {roc_auc_score(true[:, i], pred[:, i])}")
    print(f"Evaluate loss {total_loss / len(iterator)}")

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
{'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
{'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
EPOCH_NUM = 1
# triangular learning rate, linearly grows untill half of first epoch, then linearly decays 
warmup_steps = int(0.5 * len(train_iterator))
total_steps = len(train_iterator) * EPOCH_NUM - warmup_steps
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=1e-8)
# scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_steps, t_total=total_steps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

In [ ]:
for i in range(EPOCH_NUM):
    print('=' * 50, f"EPOCH {i}", '=' * 50)
    train(model, train_iterator, optimizer, scheduler)
    evaluate(model, dev_iterator)

In [ ]:
# 保存模型
output_dir='./sim_model'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)